In [ ]:
import huggingface_hub
hf_token = '' # put your User Access Tokens here
# ابتدا login کنید
huggingface_hub.login(token=hf_token)

# سپس وضعیت ورود را بررسی کنید
!hf auth whoami

user:  AM-Nateghi


In [1]:
# ================================================================
# Text Normalization
# ================================================================
ARABIC_TO_PERSIAN = {"\u064a": "ی", "\u0643": "ک", "\u06cc": "ی"}
PERSIAN_DIGITS_TO_EN = {f"۰۱۲۳۴۵۶۷۸۹"[i]: str(i) for i in range(10)}


def normalize_text(text: str) -> str:
    """Normalize Persian text"""
    if not isinstance(text, str):
        return ""
    for ar, fa in ARABIC_TO_PERSIAN.items():
        text = text.replace(ar, fa)
    for fa, en in PERSIAN_DIGITS_TO_EN.items():
        text = text.replace(fa, en)
    text = " ".join(text.split())
    text = text.replace(" ؟", "؟").replace("،", "، ")
    return " ".join(text.split())

In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

torch.cuda.empty_cache()

cptk = "google/gemma-3-270m"

# بارگذاری tokenizer
tokenizer = AutoTokenizer.from_pretrained("../outputs/gemma3_weighted/final/")
# بارگذاری مدل پایه
base_model = AutoModelForCausalLM.from_pretrained(cptk)
# بارگذاری LoRA روی مدل پایه
model = PeftModel.from_pretrained(base_model, "../outputs/gemma3_weighted/final/")
# انتقال به GPU
model.to("cuda")
print("ok!")

/home/ai/Desktop/Fine-Tune/.env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/ai/Desktop/Fine-Tune/.env/lib/python3.12/site-packages/peft/tuners/tuners_utils.py:693: UserWarning: Model with `tie_word_embeddings=True` and the tied_target_modules=['model.embed_tokens'] are part of the adapter. This can lead to complications, for example when merging the adapter or converting your model to formats other than safetensors. See for example https://github.com/huggingface/peft/issues/2018.
  warnings.warn(


ok!


In [8]:
q_norm = normalize_text("چه نوع لذتی می‌تواند در انجام کارهای ناگفتنی مانند مراقبت از بیمار یا حمایت از آسیب‌پذیر یافت شود؟")
text = f"سوال: {q_norm}\nپاسخ: "
output = tokenizer.decode(
    model.generate(
        **tokenizer(text, return_tensors="pt").to(
            "cuda"
        ),
        max_new_tokens=256,
    )[0],
    skip_special_tokens=True
)
print(output)

سوال: چه نوع لذتی می‌تواند در انجام کارهای ناگفتنی مانند مراقبت از بیمار یا حمایت از آسیب‌پذیر یافت شود؟
پاسخ: 1.مداد و طول و ارتفاع این قسمت از بدن نیست 2.بدون این‌ها کارهای بد روی بدن نیست و بدن هم محدود به کارهای بد نیست و این کارهای بد روی بدن در یک مدت نیست 3.بدون کارهای بد بدن این بد در یک مدت نیست 4.بدون کارهای بد بدن هم مانند پرواز نیست 5.بدون پرواز در این بدن نیست 6.بدون پرواز محدود به این بدن نیست 7.بدون پرواز در این بدن هم مثل لذت یابند یا لذت بردن به لذت بردن 8.بدون پرواز به سمت خداوند در این بدن نیست 9.بدون پرواز هم مثل لذت بردن به لذت بردن 10.بدون پرواز به سمت مقام ولایت در این بدن نیست 11.بدون این لذت بخاطر اینه که فقر و نیاز نیست. 12.بدون این فقر و نیاز فقر و در دسترس نیست و به دست نیفتن و دست اندر دست پای ماست 13.بدون فقر و نیاز انسان در این فقر به دنبال لذت بردن نیست ولی به دنبال سعادت نیست 14
